<a href="https://colab.research.google.com/github/Bhekmuzi/water-usage-norm/blob/main/%5DWaterUsageAndNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install influxdb

In [ ]:
# !pip install pandas pymongo

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
# pip install influxdb
# !python --version
import numpy as np
import pandas as pd
import datetime
import time
from influxdb import InfluxDBClient
from pymongo import MongoClient
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
# from scipy.stats import entropy

# Provide the IP address, username, password, database name, RFC3339 standard time format, and create a connection client for the 'db0' database
client = InfluxDBClient('59.120.114.133', 8086, 'telegraf', 'telegraf', 'db0', 'rfc3339', timeout=10)


In [ ]:
# Function to calculate start and end times for the previous day
def calculate_previous_day_times():
    current_datetime = datetime.now()
    start_of_previous_day = current_datetime - timedelta(days=1)
    start_of_previous_day = start_of_previous_day.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_previous_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(microseconds=1)

    return start_of_previous_day, end_of_previous_day


In [ ]:
# start_time = '2024-01-01T00:00:00Z'
# end_time = '2024-01-01T23:59:59Z'

# InfluxDB query
start_time, end_time = calculate_previous_day_times()

sql_string = f'SELECT DISTINCT("value") AS value FROM mbMQTT6 WHERE "topic" = \'mbMQTT2/home2127/C2BDF8/TH20\' AND time >= \'{start_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' AND time <= \'{end_time.strftime("%Y-%m-%dT%H:%M:%SZ")}\' GROUP BY time(10s) FILL(previous) ORDER BY time ASC TZ(\'Asia/Taipei\');'

# sql_string = 'SELECT distinct("value") as value FROM mbMQTT6 WHERE "topic" = \'mbMQTT2/home2127/C2BDF8/TH20\' and time >= \'' + start_time + '\' and time <= \'' + end_time + '\' GROUP BY time(10s) FILL(previous) ORDER by time ASC tz(\'Asia/Taipei\');'

result = client.query(sql_string) #

df = pd.DataFrame(result['mbMQTT6'])# Convert the result to a Pandas DataFrame

df['time'] = pd.to_datetime(df['time'])# Convert 'timestamp' to datetime format

df['time'] = pd.to_datetime(df['time']).dt.tz_convert('Asia/Taipei')

df.set_index('time', inplace=True)# Set 'time' as the index

# Create a DataFrame with the desired time intervals (10-second frequency)
start_time = df.index.min()
end_time = df.index.max()
time_intervals = pd.date_range(start=start_time, end=end_time, freq='10S')

# Create a DataFrame with all time intervals
all_time_intervals_df = pd.DataFrame(index=time_intervals)

# Merge the original DataFrame with the DataFrame containing all time intervals
merged_df = all_time_intervals_df.merge(df, left_index=True, right_index=True, how='left')

# Forward-fill missing values
merged_df['value'].fillna(method='ffill', inplace=True)

merged_df['volume_liters'] = pd.to_numeric(merged_df['value'], errors='coerce') / 100 # divide 'value' by 100000 and multiply by 1000


In [ ]:

homes_data = {
    'home2127':pd.DataFrame(result['mbMQTT6']),
    'home2128':pd.DataFrame(result['mbMQTT6'])
}

In [ ]:
# Function to fill missing values for a given home
def fill_missing_values(home_data):
    home_data['time'] = pd.to_datetime(home_data['time'])
    home_data.set_index('time', inplace=True)
    expected_time_intervals = pd.date_range(start=home_data.index.min(), end=home_data.index.max(), freq='10S')
    home_data = home_data.reindex(expected_time_intervals)
    home_data['value'] = home_data['value'].fillna(method='pad')
    home_data.reset_index(inplace=True)
    return home_data

In [ ]:
# Loop through each home and fill missing values
filled_homes_data = {}
for home_name, home_df in homes_data.items():
    filled_homes_data[home_name] = fill_missing_values(home_df)

# Display the resulting filled DataFrames
for home_name, filled_home_data in filled_homes_data.items():
    print(f"\nFilled Data for {home_name}:\n{filled_home_data}")

In [ ]:
# Loop through each home and perform the division and multiplication
transformed_homes_data = {}
for home_name, filled_home_data in filled_homes_data.items():
    # Convert 'value' column to numeric
    filled_home_data['value'] = pd.to_numeric(filled_home_data['value'], errors='coerce')

    # Perform division and multiplication
    filled_home_data['value'] = filled_home_data['value'] / 100000 * 1000

    # Store the transformed DataFrame
    transformed_homes_data[home_name] = filled_home_data

# Display the resulting transformed DataFrames
for home_name, transformed_home_data in transformed_homes_data.items():
    print(f"\nTransformed Data for {home_name}:\n{transformed_home_data}")


In [ ]:
# Loop through each home and replace NaN with 0 in the 'value_diff' column
filled_nan_homes_data = {}
for home_name, differenced_home_data in differenced_homes_data.items():
    # Replace NaN with 0 in the 'value_diff' column
    differenced_home_data['value_diff'] = differenced_home_data['value_diff'].fillna(0)

    # Store the DataFrame with filled NaN values
    filled_nan_homes_data[home_name] = differenced_home_data

# Display the resulting DataFrames with filled NaN values
for home_name, filled_nan_home_data in filled_nan_homes_data.items():
    print(f"\nData for {home_name} with NaN filled with 0:\n{filled_nan_home_data}")



Data for home2127 with NaN filled with 0:
                         index     value  value_diff
0    2024-01-02 00:00:00+00:00  267.9127         0.0
1    2024-01-02 00:00:10+00:00  267.9127         0.0
2    2024-01-02 00:00:20+00:00  267.9127         0.0
3    2024-01-02 00:00:30+00:00  267.9127         0.0
4    2024-01-02 00:00:40+00:00  267.9127         0.0
...                        ...       ...         ...
8635 2024-01-02 23:59:10+00:00  269.0890         0.0
8636 2024-01-02 23:59:20+00:00  269.0890         0.0
8637 2024-01-02 23:59:30+00:00  269.0890         0.0
8638 2024-01-02 23:59:40+00:00  269.0890         0.0
8639 2024-01-02 23:59:50+00:00  269.0890         0.0

[8640 rows x 3 columns]

Data for home2128 with NaN filled with 0:
                         index     value  value_diff
0    2024-01-02 00:00:00+00:00  267.9127         0.0
1    2024-01-02 00:00:10+00:00  267.9127         0.0
2    2024-01-02 00:00:20+00:00  267.9127         0.0
3    2024-01-02 00:00:30+00:00  267.9127 

In [ ]:
# Loop through each home and filter values less than 0.2 in the 'value_diff' column
filtered_homes_data = {}
for home_name, filled_nan_home_data in filled_nan_homes_data.items():
    # Filter values less than 0.2 in the 'value_diff' column
    filtered_home_data = filled_nan_home_data[filled_nan_home_data['value_diff'] < 0.2]

    # Store the filtered DataFrame
    filtered_homes_data[home_name] = filtered_home_data

# Display the resulting filtered DataFrames
for home_name, filtered_home_data in filtered_homes_data.items():
    print(f"\nFiltered Data for {home_name}:\n{filtered_home_data}")



Filtered Data for home2127:
                         index     value  value_diff
0    2024-01-02 00:00:00+00:00  267.9127         0.0
1    2024-01-02 00:00:10+00:00  267.9127         0.0
2    2024-01-02 00:00:20+00:00  267.9127         0.0
3    2024-01-02 00:00:30+00:00  267.9127         0.0
4    2024-01-02 00:00:40+00:00  267.9127         0.0
...                        ...       ...         ...
8635 2024-01-02 23:59:10+00:00  269.0890         0.0
8636 2024-01-02 23:59:20+00:00  269.0890         0.0
8637 2024-01-02 23:59:30+00:00  269.0890         0.0
8638 2024-01-02 23:59:40+00:00  269.0890         0.0
8639 2024-01-02 23:59:50+00:00  269.0890         0.0

[8640 rows x 3 columns]

Filtered Data for home2128:
                         index     value  value_diff
0    2024-01-02 00:00:00+00:00  267.9127         0.0
1    2024-01-02 00:00:10+00:00  267.9127         0.0
2    2024-01-02 00:00:20+00:00  267.9127         0.0
3    2024-01-02 00:00:30+00:00  267.9127         0.0
4    2024-01-02 

In [ ]:
filtered_home_data

,index,value,value_diff
0,2024-01-02 00:00:00+00:00,267.9127,0.0
1,2024-01-02 00:00:10+00:00,267.9127,0.0
2,2024-01-02 00:00:20+00:00,267.9127,0.0
3,2024-01-02 00:00:30+00:00,267.9127,0.0
4,2024-01-02 00:00:40+00:00,267.9127,0.0
...,...,...,...
8635,2024-01-02 23:59:10+00:00,269.0890,0.0
8636,2024-01-02 23:59:20+00:00,269.0890,0.0
8637,2024-01-02 23:59:30+00:00,269.0890,0.0
8638,2024-01-02 23:59:40+00:00,269.0890,0.0


In [ ]:
merged_df['volume_liters'] = merged_df['volume_liters'].diff()

merged_df['volume_liters'].fillna(0, inplace=True) # Replace NaN values with 0 in the 'flow' and 'value' columns

# merged_df['volume_liters'] = pd.to_numeric(merged_df['volume_m^3'], errors='coerce') * 1000

merged_df.loc[merged_df['volume_liters'] < 0.2, 'volume_liters'] = 0 # Filter values less than 0.2


# Remove 'Time' column as the index and revert to the default integer index
merged_df.reset_index(drop=False, inplace=True)

# Rename the 'Time_header' column to 'time'
merged_df = merged_df.rename(columns={'index': 'time'})
# merged_df

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
from scipy.stats import entropy

def process_events(df):
    start_time = None
    end_time = None
    consecutive_zeros = 0
    total_time = timedelta()
    total_vol = 0
    num_records = 0

     # Create a new DataFrame to store event information
    event_df = pd.DataFrame(columns=['Start Time', 'End Time'])

    for index, row in df.iterrows():
        time = row['time']
        vol = max(0, float(row['volume_liters']))

        if vol != 0:
            if start_time is None:
                start_time = time - timedelta(seconds=60)
            consecutive_zeros = 0
        else:
            consecutive_zeros += 1
            if start_time is not None and consecutive_zeros == 6:
                end_time = time
                # print(f"Event start_time: {start_time}, end_time: {end_time}")

                # Filter volume values between start_time and end_time
                event_data = df[(df['time'] >= start_time) & (df['time'] <= end_time)]

                # Calculate total time, total volume, and coefficient of variation
                time1 = start_time + timedelta(seconds=60)
                time2 = end_time - timedelta(seconds=60)
                event_data1 = df[(df['time'] >= time1) & (df['time'] <= time2)]

                event_df = pd.concat([event_df, pd.DataFrame({
                    'Start Time': [time1],
                    'End Time': [time2],
                })], ignore_index=True)

                start_time = None  # Reset start_time for the next event

    if event_df.empty:
        print("No events detected.")
        return pd.DataFrame()  # Return an empty DataFrame if no events are detected

    return event_df

In [ ]:
event_df = process_events(merged_df)

# Calculate the duration of each event
event_df['Duration'] = (event_df['End Time'] - event_df['Start Time']).dt.total_seconds()

event_df = event_df[event_df['Duration'] != 0]

# Filter events with duration less than 20 seconds
event_atleast_20_df = event_df[event_df['Duration'] >= 10]

# event_df = process_events(merged_df)
merged_df['usage'] = 0

# Iterate through rows in df1 and mark corresponding rows in df as 1
for index, row in event_atleast_20_df.iterrows():
    mask = (merged_df['time'] >= row['Start Time']) & (merged_df['time'] <= row['End Time'])
    merged_df.loc[mask, 'usage'] = 1

vol_df = merged_df[['time', 'volume_liters']].copy()
usage_df = merged_df[['time', 'usage']].copy()

usage_df.set_index('time', inplace=True)# Set 'time' as the index
vol_df.set_index('time', inplace=True)# Set 'time' as the index

In [ ]:
data_dict = merged_df.to_dict(orient='records')

In [ ]:
# Replace 'your_database' and 'your_collection' with your actual database and collection names
client = MongoClient('mongodb://localhost:27017/')
db = client['Taipower']
collection = db['SmartWaterMeterActiveHistory']

In [ ]:
collection.insert_many(data_dict)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import variation, entropy

# Iterate over events
for i, event in event_df.iterrows():
    start_time_event = event['Start Time']
    end_time_event = event['End Time']

    # Filter 'merged_df' DataFrame for rows within the time range of the current event
    filtered_data = merged_df[(merged_df['time'] >= start_time_event) & (merged_df['time'] <= end_time_event)]

    # Calculate duration, total volume, coefficient of variation, and entropy in filtered_data
    duration = (filtered_data['time'].max() - filtered_data['time'].min()).total_seconds()
    total_volume = filtered_data['volume_liters'].sum()
    coef_variation = variation(filtered_data['volume_liters'])
    vol_distribution = np.histogram(filtered_data['volume_liters'], bins=10)[0]
    ent = entropy(vol_distribution)

    # Plot volume values between start_time and end_time
    plt.figure(figsize=(6, 1))
    # Plot 'usage' for the current event
    plt.bar(filtered_data.index, filtered_data['volume_liters'], label=f'Volume {i + 1}')

    # Set plot labels and title
    plt.xlabel('Time')
    plt.ylabel('Volume (Liters)')
    plt.title(f'Date: {start_time_event.date()}, Time Range: {start_time_event.time()}-{end_time_event.time()}\n'
              f'Duration: {duration:.2f} sec, Vol: {total_volume:.2f} L, '
              f'CV: {coef_variation:.2f}, Entropy: {ent:.2f}')

    # Set y-axis limits
    plt.ylim([-0.5, 4])
    # Rotate x-axis labels for better readability (optional)
    # plt.xticks(rotation=45)
    plt.legend()
    plt.grid()

    plt.show()


In [ ]:
tm_vol = merged_df[['time', 'volume_liters']]
tm_usage = merged_df[['time', 'usage']]

# Convert the 'time' column to datetime
tm_vol['time'] = pd.to_datetime(tm_vol['time'])
tm_usage['time'] = pd.to_datetime(tm_vol['time'])

# Create new columns for the day and time of day
tm_vol['day'] = tm_vol['time'].dt.date
tm_usage['day'] = tm_usage['time'].dt.date
tm_vol['time_of_day'] = tm_vol['time'].dt.time
tm_usage['time_of_day'] = tm_usage['time'].dt.time

# Pivot the DataFrame to reshape 'norm_vol' values into columns for each day and rows for each time of day
vol_day = tm_vol.pivot_table(index='time_of_day', columns='day', values='volume_liters', aggfunc='first')
vol_usage = tm_usage.pivot_table(index='time_of_day', columns='day', values='usage', aggfunc='first')
vol_day = vol_day.fillna(0)
vol_usage = vol_usage.fillna(0)
# vol_day
# vol_usage

In [ ]:
# Resample the DataFrame to 15-minute intervals and sum the values within each interval
resampled_df = usage_df.resample('15T').max()
resampled_vol_df = vol_df.resample('15T').sum()
resampled_df['volume_liters'] = resampled_vol_df['volume_liters']

# Reset the index to add a numerical index
resampled_df = resampled_df.reset_index()

# Convert the 'time' column to datetime
resampled_df['time'] = pd.to_datetime(resampled_df['time'])

# Create new columns for the day and time of day
resampled_df['day'] = resampled_df['time'].dt.date

resampled_df['time_of_day'] = resampled_df['time'].dt.time

# resampled_df.loc[df['norm_vol'] < 0.1, 'norm_vol'] = 0 # Filter: set volume < 0.1 to 0.
# resampled_df

In [ ]:
# Pivot the DataFrame to reshape 'norm_vol' values into columns for each day and rows for each time of day
vol_day_df = resampled_df.pivot_table(index='time_of_day', columns='day', values='volume_liters', aggfunc='first')
vol_day_df = vol_day_df.fillna(0)
# vol_day_df

NameError: ignored

In [ ]:
vol_day_df

NameError: ignored

In [ ]:
# Create a new DataFrame with values set to 0 if less than 0.1 and 1 if greater than 0.1
usage_df = pd.DataFrame(vol_day_df.applymap(lambda x: 0 if x == 0 else 1))

# Convert column names to datetime objects if needed
vol_day.columns = pd.to_datetime(vol_day.columns)
vol_day_df.columns = pd.to_datetime(vol_day_df.columns)

# Convert column names to datetime objects if needed
usage_df.columns = pd.to_datetime(usage_df.columns)

# Choose the datetime for which you want to create the subplot
selected_datetime = pd.to_datetime('2023-12-14')  # Replace with the desired datetime

# Set the desired width of the bars
bar_width = 1.0

plt.figure(figsize=(22, 3))

# Plot line and bar chart for the selected datetime
volume_day = vol_day[selected_datetime]
volume_min = vol_day_df[selected_datetime]
x_volume = range(len(volume_day))
x_vol = range(len(volume_min))
usage_day = usage_df[selected_datetime]
x_usage = range(len(usage_day))
threshold = [0] * 8640

# Line plot
# plt.plot(df['time'], df['vol'], label='Volume')
plt.plot(volume_day.index, volume_day, label=f'Water usage - {selected_datetime}')
# plt.plot(x_volume, threshold, color='orange', label=f'Threshold filter')
plt.ylabel('Volume (L/10sec)')
plt.ylim(-1, 5)

# # Show the plot
plt.show()


In [ ]:
# Set the desired width of the bars
bar_width = 1.0

threshold1 = [0] * 96

# Create a subplot with 2 rows and 1 column
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20, 6), sharex=True)

# Line plot
axes[0].plot(x_vol, volume_min, label=f'Water usage - {selected_datetime}')
axes[0].plot(x_vol, threshold1, color='orange', label=f'Threshold filter')
axes[0].set_ylabel('Volume (L/15min)')
axes[0].set_ylim(-1, 70)
axes[0].legend()

# Bar plot
bar_positions = [x + bar_width / 2 for x in range(len(vol_day_df))]
axes[1].bar(bar_positions, usage_day, width=bar_width)
axes[1].set_ylabel('Usage')
axes[1].set_xticks([x + bar_width / 2 for x in range(len(vol_day_df))])
axes[1].set_xticklabels(vol_day_df.index, rotation=45, ha='right')
axes[1].set_xlabel('Time of Day')
axes[1].legend()

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:

# fig, ax = plt.subplots(figsize=(20, 2))

# # Bar plot
# bar_positions = [x + bar_width / 2 for x in range(len(vol_day_df))]
# bar_width = 1.0  # Adjust the bar width as needed

# plt.bar(bar_positions, usage_day, width=bar_width, label='Usage')

# # Customize plot
# plt.ylabel('Usage')
# plt.xticks([x + bar_width / 2 for x in range(len(vol_day_df))], vol_day_df.index, rotation=45, ha='right')
# plt.xlabel('Time of Day')

# # Add legend to the plot
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
date_score = '2023-12-12'
date_norm = '2023-12-11'


# Calculate the rolling average for the previous 14 days
rolling_avg = usage_df.rolling(window=28, axis=1).mean().iloc[:, 27:]

# Multiply by 100
two_week_norm = rolling_avg * 100

two_week_norm.columns = pd.to_datetime(two_week_norm.columns)

# norm = two_week_norm.iloc[:, NDate]
norm = two_week_norm[date_norm]
x_norm = range(len(norm))
# usage = binary_df[date_score]
# usage = usage_df.iloc[:, BDate]
usage = usage_df[date_score]
x_usage = range(len(usage))

In [ ]:

# Sum the columns
norm_sum= two_week_norm.sum()
active_sum = usage_df.sum()

active_score = usage_df.sum()/96 * 100
norm_score = norm_sum/96

active_score = usage_df.sum()/96 * 100

# Sort DataFrame in descending order by column 'A'
norm_sorted = two_week_norm.sort_values(by=two_week_norm.columns[0], ascending=True)

# Find the index to split the DataFrame into two halves
split_index = len(norm_sorted) // 2

# Split the DataFrame into the first and second halves
first_half = norm_sorted.iloc[:split_index, :]
second_half = norm_sorted.iloc[split_index:, :]

# Calculate the average for each half
low_norm = first_half.mean()
high_norm = second_half.mean()


In [ ]:
two_week_norm

In [ ]:
norms = pd.DataFrame(high_norm)
# Assuming you want to rename the column named 'column1' to 'new_column_name'
new_column_name = 'high_norm'
norms['norm'] = norm_score
norms['low_norm'] = low_norm
norms['score'] = active_score
norms = norms.rename(columns={'column1': new_column_name})
# norms

In [ ]:
# Set the desired width of the bars
bar_width = 1.0

norma_score = norms['norm'][date_norm]

# Create a subplot with 2 rows and 1 column
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 5), sharex=True)
usage_datetime = date_score

# Bar chart for 'Norm'
usage = usage_df[usage_datetime]

# Assuming usage_df is your DataFrame
cor_coef = usage_df[usage_datetime].corr(norm)

# Bar chart for 'Usage'
# norm = one_month_norm.iloc[:, 1]
x_norm = range(len(norm))
axes[0].bar(x_norm, usage, width=bar_width, label=f'water usage - {usage_datetime}')
axes[0].set_ylabel('Usage')
axes[0].legend()

# Add a grid
axes[0].grid(True, linestyle='--', alpha=0.7)
axes[0].set_title(f'Active_score, correlation coefficient: {round(active_score[date_score],3)}, {round(cor_coef,3)}')  # Set title for the second subplot


norm_datetime = date_norm
x_usage = range(len(usage))
bar_positions = [x + bar_width / 2 for x in x_usage]
axes[1].bar(bar_positions, norm, width=bar_width, label=f'two-week Norm - {norm_datetime}')
axes[1].set_ylabel('Norm (%)')
# axes[1].set_xticks([x + bar_width / 2 for x in range(len(usage_df))])
# Set x-axis ticks and labels
# Assuming usage_df.index is a DatetimeIndex
# hourly_ticks = pd.date_range(start=usage_df.index.min(), end=usage_df.index.max(), freq='1H')
# Assuming usage_df.index is a DatetimeIndex
start_time = pd.Timestamp(usage_df.index.min().strftime('%Y-%m-%d') + ' 00:00')
end_time = pd.Timestamp(usage_df.index.max().strftime('%Y-%m-%d') + ' 23:45')
# hourly_ticks = pd.date_range(start=start_time, end=end_time, freq='1H')
hourly_ticks = pd.date_range(start=start_time, end=end_time, freq='1H15T')
# axes[1].set_xticks([x + bar_width / 2 for x in range(len(usage_df))])
# axes[1].set_xticks([x + bar_width / 2 for x in range(len(usage_df))][::4])  # Adjust the stride to match the 1-hour interval
custom_ticks_positions = [x + bar_width / 2 for x in range(len(usage_df)) if x % 5 == 0]  # Adjust the condition to match 1H15T intervals
axes[1].set_xticks(custom_ticks_positions)


axes[1].set_xticklabels([hourly.strftime('%H:%M') for hourly in hourly_ticks])#, rotation=45, ha='right')  # Set x-axis ticks and labels

# axes[1].set_xticklabels(usage_df.index, rotation=45, ha='right')
axes[1].set_xlabel('Time of Day')

# Set y-axis limits
axes[1].set_ylim(0, 100)  # Set the desired y-axis limits
axes[1].set_title(f'High_norm, Norm, Low_norm: {round(norms[0][date_norm],3)}, {round(norma_score,3)}, {round(low_norm[date_norm],3)}')  # Set title for the second subplot

# Add legend to the plots
axes[1].legend()


# Add a grid
axes[1].grid(True, linestyle='--', alpha=0.7)

# Adjust layout to avoid label clipping
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Set the desired width of the bars
bar_width = 1.0

usage_datetime = date_score

# Bar chart for 'Usage'
usage = usage_df[usage_datetime]

# Assuming usage_df is your DataFrame
cor_coef = usage_df[usage_datetime].corr(norm)

# Create a figure
fig, ax = plt.subplots(figsize=(10, 2))

# Bar chart for 'Usage'
x_usage = range(len(usage))
ax.bar(x_usage, usage, width=bar_width, label=f'Water Usage - {usage_datetime}')
ax.set_ylabel('Usage')
ax.legend()
ax.set_title(f'Active Score, Correlation Coefficient: {round(active_score[date_score], 3)}, {round(cor_coef, 3)}')

# Set x-axis ticks and labels
start_time = pd.Timestamp(usage_df.index.min().strftime('%Y-%m-%d') + ' 00:00')
end_time = pd.Timestamp(usage_df.index.max().strftime('%Y-%m-%d') + ' 23:45')
hourly_ticks = pd.date_range(start=start_time, end=end_time, freq='1H15T')
custom_ticks_positions = [x + bar_width / 2 for x in range(len(usage_df)) if x % 5 == 0]
ax.set_xticks(custom_ticks_positions)
ax.set_xticklabels([hourly.strftime('%H:%M') for hourly in hourly_ticks])#, rotation=45, ha='right')

# Set y-axis limits
ax.set_ylim(0, usage_df[usage_datetime].max())  # Adjust the y-axis limits as needed

# Set x-axis label
ax.set_xlabel('Time of Day')

# Adjust layout to avoid label clipping
plt.tight_layout()
plt.grid()

# Show the plot
plt.show()
